<a href="https://colab.research.google.com/github/mariliabrito/gunviolence/blob/main/Viol%C3%AAncia_Escolar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="TAIL"><img src="https://i.imgur.com/GRTE0OG.png" width="150" title="Tail Logo" /></a>

# <font color ="#00b8ff">Análise de Violência Escolar</font>
## <font color ="#00b8ff">Trainees - 2022.2</font>

## <font color ="#00b8ff">1. Introdução</font>
**Equipe:** Marília Brito, Jonas, Gisele, Hermano Neto, João Victor Dantas, Maria Eduarda Bandeira, Raffael Leobino

 FOTO 

## Introdução 

Após a ocorrência de dois massacres em território brasileiro no intervalo de uma semana, o tema em questão recebeu grande destaque. O país alarmou-se diante de ameaças e possíveis novos ataques. Diante dessa situação, levantou-se diversos questionamentos quanto à natureza dessas ocorrências.

Segundo um mapeamento da Universidade Estadual de Campinas (Unicamp) sobre casos de ataques em escolas por alunos ou ex-alunos, o primeiro episódio foi registrado em 2002, quando um adolescente de 17 anos disparou contra duas colegas dentro da sala de aula de uma escola particular de Salvador. Desde esse caso, foram contabilizados 22 ataques, 13 deles realizados nos últimos dois anos.

A fim de compreender o fenômeno, suas causas e possíveis formas de proteção, a equipe de Trainees resolveu analisar um dataset de violência escolar. Foi escolhida uma base norte-americana, visto que essa totaliza uma maior quantidade de dados, 2069 ocorrências, de 1970 até junho de 2022, e possui mais variáveis para estudo. 

O dataset foi obtido através do Center for Homeland, Defense
and Security (School Shooting Safety Compendium) e todos os casos apresentam link da nóticia/registro, de forma a garantir sua confiabilidade.

## Objetivos gerais 
Extrair informações relevantes sobre a temática
de violência nas escolas envolvendo violentadores;

## Objetivos específicos
* Observar tendência ao longo do tempo;
* Identificar perfil do violentador e da vítima;
* Extrair possíveis fatores de risco e proteção;
* Realizar análise preditiva.

### Colunas do dataset:

In [ ]:
import pandas as pd

In [ ]:
# Importando 4 abas do arquivo

df_incid = pd.read_excel("https://github.com/mariliabrito/gunviolence/blob/main/Public%20v3.1%20K-12%20School%20Shooting%20Database.xlsx?raw=true", sheet_name=1)
df_shoot = pd.read_excel("https://github.com/mariliabrito/gunviolence/blob/main/Public%20v3.1%20K-12%20School%20Shooting%20Database.xlsx?raw=true", sheet_name=2)
df_victm = pd.read_excel("https://github.com/mariliabrito/gunviolence/blob/main/Public%20v3.1%20K-12%20School%20Shooting%20Database.xlsx?raw=true", sheet_name=3)
df_weapn = pd.read_excel("https://github.com/mariliabrito/gunviolence/blob/main/Public%20v3.1%20K-12%20School%20Shooting%20Database.xlsx?raw=true", sheet_name=4)

HTTPError: ignored

## <font color ="#00b8ff">2. Limpeza</font>

Realizar limpeza e uniformização do dataset. Também é possível juntar datasets pela coluna incidentid.

## <font color ="#00b8ff">3. Análise Descritiva</font>

Realizar análise do dataset, mostrar série histórica, trazer visualizações, etc

### <font color ="#00b8ff">3.1 Série histórica</font>

### <font color ="#00b8ff">3.2 Análise dos Incidentes</font>

### <font color ="#00b8ff">3.3 Análise do Atirador</font>

### <font color ="#00b8ff">3.4 Análise das Vítimas</font>

### <font color ="#00b8ff">3.5 Análise das Armas</font>